# Classification and Clustering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections

import utils_vectoriser

### 1. Create training and evaluation sets

In [ ]:
# do train-test this at each author level, before merging and exporting as SVMlight

In [2]:
test = pd.read_pickle("../2_Preprocessing/processeddata/df_pickles/a95_df.pickle")
test.head(2)

,authornum,concreteness,lemmas,ne_dates,ne_persons,ne_places,parse_NP,parse_S,parse_VP,pos_ADJ,...,pos_PUNCT,pos_SCONJ,pos_SPACE,pos_SYM,pos_VERB,pos_X,poswordpairs,sentences,tokens,sent_length
0,a95,0.25,"[-PRON-, be, pleased, with, the, word, .]",[],[],[],2,1,1,1,...,1,0,0,0,1,0,"[(PRON, I), (VERB, am), (ADJ, pleased), (ADP, ...",I am pleased with the word.,"[i, am, pleased, with, the, word]",6
1,a95,2.25,"[[, 1, ], this, be, immediately, follow, by, a...",[],[],[],7,4,3,1,...,5,0,0,0,3,0,"[(PUNCT, [), (PUNCT, 1), (PUNCT, ]), (DET, Thi...",[1] This was immediately followed by an interv...,"[1, this, was, immediately, followed, by, an, ...",23


In [12]:
test_dict = utils_vectoriser._compute_frequency(test["lemmas"])

In [13]:
x,y = utils_vectoriser._vectoriser(test_dict)

In [14]:
len(y)

1882

In [ ]:
# load the data set. every single author_df, merged.  
# extract the features 
# tokens with TFID normalisation on the raw counts  - run the tokeniser and then add to the dataframe 
# NER with TFID normalisation on the raw counts 
# check correlation between features and 

# saving to SVM 
# train_tokens, test_tokens
# train_ner, test_ner



In [ ]:
# write to svmlight 
sklearn.datasets.dump_svmlight_file()

def _save_to_file( X, y, file):
    '''
    Save X and y into file, using the svmlight format.
    X: sparse matrix (samples)
    y: numpy array (labels)
    file: path 
    '''
    dump_svmlight_file( X, y, file )


### 2. Multilabel Classification 

In [ ]:
X, Y = make_multilabel_classification(n_classes=5, n_labels=5,
                                      allow_unlabeled=True,
                                      random_state=1)

In [ ]:
X[0]

In [ ]:
# we need to get the labels encoded like this for multilabel classification
# step 1, find out how many unique labels we have in the corpus
# have a look at the distribution of the label. are there significant outliers. if few, consider removing them
# else, we may consider augmenting datapoints with SMOTE 
Y[0]

In [ ]:
>>> from sklearn import datasets
>>> from sklearn.multiclass import OneVsRestClassifier
>>> from sklearn.linear_model import LogisticRegression
from sklearn.
>>> iris = datasets.load_iris()
>>> X, y = iris.data, iris.target

### 3. Clustering with Graph propagation extension

In [ ]:
# i think we can try this. since our data set is not very big. we could use our initial clusters as a feature and retrain 
# the classifier 

# https://arxiv.org/pdf/1709.05634.pdf 